In [1]:
import numpy as np
from keras.models import load_model, Model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Features from pretrained feature extractors

In [2]:
def get_indices(net_name, mode):
    filenames = np.load('../input/features/' + net_name + '/filenames_' + mode +'.npy')
    indices = np.argwhere(np.array(list(map(lambda fn: fn.split('/')[1][0] != '_', filenames))) == True)
    indices = indices.reshape((1, indices.shape[0]))[0]
    return indices

def get_features_from_nets(net_names, mode, steps):
    indices = get_indices('res_net', mode)
    classes = np.load('../input/features/res_net/classes_' + mode + '_make-model.npy')[indices]
    classnames = np.load('../input/features/res_net/classnames_' + mode + '_make-model.npy')[indices]
    filenames = np.load('../input/features/res_net/filenames_' + mode + '.npy')[indices]
    features = []
    for net_name in net_names:
        net_features = np.load('../input/features/' + net_name + '/features_' + mode +'.npy')[indices]
        features.append(net_features)
    features = np.concatenate(features, axis=1)
    features = np.expand_dims(features, axis=1)
    features = np.repeat(features, steps, axis=1)
    return features, classes, classnames, filenames

In [3]:
features_train, classes_train, classnames_train, filenames_train = get_features_from_nets(['res_net', 'inception', 'xception'], 'train', 3)
features_dev, classes_dev, classnames_dev, filenames_dev = get_features_from_nets(['res_net', 'inception', 'xception'], 'dev', 3)

## Features from LSTM

In [4]:
model = load_model('../input/model/car-identification.h5')
model = Model(inputs=model.input, outputs=model.layers[6].output)

/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
predictions_train = model.predict(features_train, verbose=1)
predictions_dev = model.predict(features_dev, verbose=1)

996/996 [==============================] - 5s 5ms/step


## Divide features

In [6]:
features = np.concatenate((predictions_train, predictions_dev), axis=0)
classes = np.concatenate((classes_train, classes_dev), axis=0)
classnames = np.concatenate((classnames_train, classnames_dev), axis=0)
filenames = np.concatenate((filenames_train, filenames_dev), axis=0)

In [7]:
print(features.shape, classes.shape, classnames.shape, filenames.shape)

(4833, 1024) (4833,) (4833,) (4833,)


In [8]:
test_classes = np.random.choice(np.unique(classes), 5)
print(test_classes)

[ 7 11 25 21 30]


In [11]:
train_indices = np.argwhere(np.in1d(classes, test_classes) == False)
train_indices = train_indices.reshape((train_indices.shape[0],))
dev_indices = np.array([], dtype='int')
test_indices = np.array([], dtype='int')

for cl in test_classes:
    cl_dev_indices, cl_test_indices = train_test_split(np.argwhere(classes == cl), test_size=0.2, random_state=1)
    dev_indices = np.append(dev_indices, cl_dev_indices)
    test_indices = np.append(test_indices, cl_test_indices)

In [12]:
print(train_indices.shape, dev_indices.shape, test_indices.shape)

(4111,) (576,) (146,)


In [13]:
features_train_to_save = features[train_indices]
classes_train_to_save = classes[train_indices]
classnames_train_to_save = classnames[train_indices]
filenames_train_to_save = filenames[train_indices]

features_dev_to_save = features[dev_indices]
classes_dev_to_save = classes[dev_indices]
classnames_dev_to_save = classnames[dev_indices]
filenames_dev_to_save = filenames[dev_indices]

features_test_to_save = features[test_indices]
classes_test_to_save = classes[test_indices]
classnames_test_to_save = classnames[test_indices]
filenames_test_to_save = filenames[test_indices]

In [14]:
np.save('../input/features/siamese/features_train.npy', features_train_to_save)
np.save('../input/features/siamese/classes_train.npy', classes_train_to_save)
np.save('../input/features/siamese/classnames_train.npy', classnames_train_to_save)
np.save('../input/features/siamese/filenames_train.npy', filenames_train_to_save)

np.save('../input/features/siamese/features_dev.npy', features_dev_to_save)
np.save('../input/features/siamese/classes_dev.npy', classes_dev_to_save)
np.save('../input/features/siamese/classnames_dev.npy', classnames_dev_to_save)
np.save('../input/features/siamese/filenames_dev.npy', filenames_dev_to_save)

np.save('../input/features/siamese/features_test.npy', features_test_to_save)
np.save('../input/features/siamese/classes_test.npy', classes_test_to_save)
np.save('../input/features/siamese/classnames_test.npy', classnames_test_to_save)
np.save('../input/features/siamese/filenames_test.npy', filenames_test_to_save)